In [ ]:
import arcade
import math
import random

SCREEN_WIDTH, SCREEN_HEIGHT = 800, 600
TITLE = "Avoid the Enemies"

SAFE_W = 120 # width of the safe zones
ENEMY_MIN_S, ENEMY_MAX_S = 120, 220
SAFE_ZONE_MAX_TIME = 15.0 # max seconds to allow in safe zone

class KeyboardDemo(arcade.Window):
    def __init__(self):
        super().__init__(SCREEN_WIDTH, SCREEN_HEIGHT, TITLE)
        arcade.set_background_color(arcade.color.PALE_SPRING_BUD)

        # Player
        self.player_list = arcade.SpriteList()
        self.player = arcade.Sprite(":resources:images/animated_characters/female_person/femalePerson_idle.png", scale=1.0)
        self.player.center_x, self.player.center_y = 80, SCREEN_HEIGHT // 2
        self.player_list.append(self.player)

        # movement and speed
        self.speed = 220.0
        self.vx = 0.0
        self.vy = 0.0

        # key states
        self.left_down = False
        self.right_down = False
        self.up_down = False
        self.down_down = False
        self.sprint_down = False

        # Enemies, score, state
        self.enemy_list = arcade.SpriteList()
        self.score = 0
        self.last_zone = 'left'
        self.game_over = False

        # Safe zone timer
        self.safe_zone_timer = 0.0
        self.in_safe_zone = False
        self.current_zone = None

        self._spawn_enemy()

    def _spawn_enemy(self):
        enemy = arcade.Sprite(":resources:images/space_shooter/meteorGrey_small1.png", scale=0.8)
        enemy.center_x = random.randint(SAFE_W + 40, SCREEN_WIDTH - SAFE_W - 40)
        enemy.center_y = random.randint(40, SCREEN_HEIGHT - 40)
        angle = random.random() * 2 * math.pi
        speed = random.uniform(ENEMY_MIN_S, ENEMY_MAX_S)
        enemy.change_x = math.cos(angle) * speed
        enemy.change_y = math.sin(angle) * speed
        self.enemy_list.append(enemy)

    def _reset(self):
        self.player.center_x, self.player.center_y = 80, SCREEN_HEIGHT // 2
        self.vx = self.vy = 0.0
        self.enemy_list = arcade.SpriteList()
        self.score = 0
        self.last_zone = "left"
        self.game_over = False
        self.left_down = self.right_down = self.up_down = self.down_down = self.sprint_down = False
        self.safe_zone_timer = 0.0
        self.in_safe_zone = False
        self.current_zone = None
        self._spawn_enemy()

    def on_draw(self):
        self.clear()

        # safe zones
        arcade.draw_rect_filled(
            arcade.rect.XYWH(SAFE_W / 2, SCREEN_HEIGHT / 2, SAFE_W, SCREEN_HEIGHT),
            arcade.color.CELADON
        )

        arcade.draw_rect_filled(
            arcade.rect.XYWH(SCREEN_WIDTH - SAFE_W / 2, SCREEN_HEIGHT / 2, SAFE_W, SCREEN_HEIGHT),
            arcade.color.LIGHT_STEEL_BLUE
        )

        self.enemy_list.draw()
        self.player_list.draw()

        # HUD
        arcade.draw_text(f"Score: {self.score}  Enemies: {len(self.enemy_list)}", 
                         16, SCREEN_HEIGHT - 58, arcade.color.BLACK, 18)
        
        if self.in_safe_zone:
            remaining = max(0, SAFE_ZONE_MAX_TIME - self.safe_zone_timer)
            arcade.draw_text(f"Safe Zone Timer: {remaining:.1f}s",
                             SCREEN_WIDTH / 2, SCREEN_HEIGHT - 32, 
                             arcade.color.DARK_RED if remaining < 5 else arcade.color.BLACK,
                             18, anchor_x="center")
        if self.game_over:
            arcade.draw_text("GAME OVER - Press R to to restart",
                             SCREEN_WIDTH / 2, SCREEN_HEIGHT - 32,
                             arcade.color.DARK_RED if remaining < 5 else arcade.color.BLACK,
                             18, anchor_x="center")
            arcade.draw_text("Final Score: {self.score}",
                             SCREEN_WIDTH / 2, SCREEN_HEIGHT / 2 - 16,
                             arcade.color.BLACK, 24, anchor_x="center")